In [2]:
import pandas as pd
import numpy as np
import os

# 1. 경로 설정
DATA_PATH = '../../data/raw/'
SUB_PATH = './submissions/'

# 2. 기준 파일 로드 (샘플 제출의 순서를 절대 기준으로 삼음)
sample_sub = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
target_ids = sample_sub.index # 혹은 'ID' 컬럼이 있다면 그 컬럼명

# 3. 모델 로드 및 강제 정렬 (Reindexing)
def load_and_align(filename):
    df = pd.read_csv(os.path.join(SUB_PATH, filename))
    # 파일의 행 순서가 샘플과 다를 리스크를 방지하기 위해 강제 재정렬
    # 만약 ID 컬럼이 있다면 df.set_index('ID').reindex(sample_sub['ID']).reset_index() 사용
    if len(df) != len(sample_sub):
        raise ValueError(f"{filename}의 행 개수가 맞지 않습니다.")
    return df['voted'].values

# [사용자 확정 파일들]
s1_voted = load_and_align('0130-1923.csv')
s2_voted = load_and_align('Pure_0781_AUC_0.77322.csv')
s7_voted = load_and_align('m7_Refined_test_preds.csv')

# 4. Double-Rank Ensemble (Index-Safe Version)
# (1) 개별 랭크 변환
r1 = pd.Series(s1_voted).rank(pct=True)
r2 = pd.Series(s2_voted).rank(pct=True)
r7 = pd.Series(s7_voted).rank(pct=True)

# (2) 가중치 결합 및 2차 랭킹 (분포 균등화)
avg_rank = (r1 * 0.3) + (r2 * 0.3) + (r7 * 0.4)
final_rank = avg_rank.rank(pct=True).values  # 0~1 사이의 완벽한 순위 정보

# (3) m1의 값 분포 주입 (Scale Restoration)
m1_values_sorted = np.sort(s1_voted)
# final_rank는 sample_sub의 순서와 1:1 대응함
indices = (final_rank * (len(m1_values_sorted) - 1)).round().astype(int)
final_voted = m1_values_sorted[indices]

# 5. 최종 검증 및 저장
submission = sample_sub.copy()
submission['voted'] = final_voted

# [검증 로직] - 아까와 같은 참사를 막기 위한 마지막 보루
print("-" * 50)
print("검증 데이터 (m1 vs Final)")
print(f"Mean 일치 여부: {abs(s1_voted.mean() - submission['voted'].mean()) < 1e-4}")
print(f"Std 일치 여부: {abs(s1_voted.std() - submission['voted'].std()) < 1e-4}")
print(f"Max 일치 여부: {s1_voted.max() == submission['voted'].max()}")
print("-" * 50)

output_name = os.path.join(SUB_PATH, '45_FINAL_TRINITY_FIXED_SAFE.csv')
submission.to_csv(output_name, index=False)
print(f"파일 생성 완료: {output_name}")

--------------------------------------------------
검증 데이터 (m1 vs Final)
Mean 일치 여부: True
Std 일치 여부: True
Max 일치 여부: True
--------------------------------------------------
파일 생성 완료: ./submissions/45_FINAL_TRINITY_FIXED_SAFE.csv
